In [7]:
import pandas as pd

In [ ]:
data = pd.read_csv("./data/clean_df_1.csv")

In [14]:
data["Dt_Customer"] = pd.to_datetime(data["Dt_Customer"])
dates = []
for value in data["Dt_Customer"]:
    value = value.date()
    dates.append(value)

print("Oldest customer join date: ", min(dates))
print("Newest customer join date:", max(dates))

# Get newest customer date
number_of_days = []
ref_date = max(dates)
for d in dates:
    delta = ref_date - d
    number_of_days.append(delta)

# Create 'Customer_For' feature
data["Customer_For"] = number_of_days
data["Customer_For"] = pd.to_numeric(data["Customer_For"], errors="coerce")

Oldest customer join date:  2012-01-08
Newest customer join date: 2014-12-06


Explore unique values in categorical features to get a clearer picture of data

In [17]:
print("Marital_Status:\n", data["Marital_Status"].value_counts(), "\n")
print("Education:\n", data["Education"].value_counts(), "\n")

Marital_Status:
 Married     857
Together    573
Single      471
Divorced    232
Widow        76
Alone         3
Absurd        2
YOLO          2
Name: Marital_Status, dtype: int64 

Education:
 Graduation    1116
PhD            481
Master         365
2n Cycle       200
Basic           54
Name: Education, dtype: int64 



Further feature engineering